In [2]:
from qiskit import QuantumCircuit
from qiskit_aer.primitives import Estimator
import matplotlib.pyplot as plt
import numpy as np
from qiskit.quantum_info import Statevector, random_clifford

from tomography import Mean_Direct_Fidelity
from tomography import NearSparseTomography 

In [3]:
NQ = 4
d = 2**NQ
shots = 10_000
simulator_ideal=Estimator(backend_options={'shots':shots,},
                            transpile_options={'optimization_level':0},
                            ) 

In [7]:
Fids = []
for _ in range(10):
    MDF = Mean_Direct_Fidelity(NQ)

    psi_circ = random_clifford( NQ ).to_circuit()
    psi_th = np.array( Statevector(psi_circ) )

    MDF.RandomMeasurements( 4**3, psi_circ, simulator_ideal )

    phi_in = np.random.rand(d) + 1j * np.random.rand(d)
    phi_in = phi_in / np.linalg.norm(phi_in)

    psi_out = NearSparseTomography( phi_in, MDF )

    fid = np.abs( np.vdot( psi_th, psi_out ) )**2 
    Fids.append( fid )

In [12]:
Fids

[0.9998733626717625,
 2.647111958524032e-05,
 0.9993543921748929,
 0.00013863573960433767,
 5.689720477314006e-05,
 0.9996469860775231,
 3.075218795725031e-05,
 0.9997842448835185,
 9.386025779226116e-07,
 0.9997775198208645]

In [10]:
np.mean( Fids )

0.4998690200483059

In [11]:
np.mean( Fids )

0.4998690200483059